# **Super-Resolution for Remote Sensing Images via Local-Global-Combined Network (2017)**
```
@article{lei2017super,
  title={Super-resolution for remote sensing images via local--global combined network},
  author={Lei, Sen and Shi, Zhenwei and Zou, Zhengxia},
  journal={IEEE Geoscience and Remote Sensing Letters},
  volume={14},
  number={8},
  pages={1243--1247},
  year={2017},
  publisher={IEEE}
}
```
1. **Model Definition**
  * Complete LGCNet model (conv1 to conv7) with exact layer shapes and skip connection (residual learning).
  * Uses ReLU after each conv.
  * Combines outputs of conv3, conv4, and conv5 using torch.cat and a 5×5 convolution.

2. **Training Process**
  * Mean squared error (MSE) loss
  * SGD optimizer (initial LR=0.1, momentum=0.9, weight decay=1e-4)
  * LR decays by 10× after 40 epochs (scheduler)
  * Gradient clipping using L2 norm
  * 80 epochs in total, batch size = 128

3. **Dataset**
  * Extracts 41×41 patches
  * LR is created by downsampling & then upsampling using bicubic
  * Training/Validation split (80/20)

4. **Metrics**
  * PSNR implemented
  * SSIM to be added optionally (using skimage.metrics.structural_similarity)

In [ ]:
import os
import math
import random
import numpy as np
import cv2
from glob import glob
from tqdm import tqdm
from PIL import Image
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T
from skimage.metrics import structural_similarity as compare_ssim

In [ ]:
class RemoteSensingDataset(Dataset):
    def __init__(self, image_paths, patch_size=41, scale=3):
        self.image_paths = image_paths
        self.patch_size = patch_size
        self.scale = scale
        self.hr_transform = T.ToTensor()

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img = Image.open(self.image_paths[idx]).convert('RGB')
        img = np.array(img)
        h, w, _ = img.shape
        ps = self.patch_size
        x = random.randint(0, w - ps - 1)
        y = random.randint(0, h - ps - 1)
        hr = img[y:y+ps, x:x+ps, :]
        lr = cv2.resize(hr, (ps//self.scale, ps//self.scale), interpolation=cv2.INTER_CUBIC)
        lr_up = cv2.resize(lr, (ps, ps), interpolation=cv2.INTER_CUBIC)
        hr = self.hr_transform(Image.fromarray(hr))
        lr_up = self.hr_transform(Image.fromarray(lr_up))
        return lr_up, hr


In [ ]:
import torch.nn.functional as F

# Define the Local-Global-Combined Network (LGCNet)
class LGCNet(nn.Module):
    def __init__(self, in_channels=3):
        super(LGCNet, self).__init__()

        # === Representation Layers ===
        # 5 convolutional layers with 3x3 kernels and 32 feature maps
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels, 32, kernel_size=3, padding=1),
            nn.ReLU(inplace=True)
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(32, 32, kernel_size=3, padding=1),
            nn.ReLU(inplace=True)
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(32, 32, kernel_size=3, padding=1),
            nn.ReLU(inplace=True)
        )
        self.conv4 = nn.Sequential(
            nn.Conv2d(32, 32, kernel_size=3, padding=1),
            nn.ReLU(inplace=True)
        )
        self.conv5 = nn.Sequential(
            nn.Conv2d(32, 32, kernel_size=3, padding=1),
            nn.ReLU(inplace=True)
        )

        # === Local-Global-Combination Layer ===
        # Concatenate conv3, conv4, conv5 -> input channels = 32 * 3 = 96
        # Then apply a 5x5 convolution to combine them
        self.lgc_merge = nn.Sequential(
            nn.Conv2d(96, 64, kernel_size=5, padding=2),
            nn.ReLU(inplace=True)
        )

        # === Reconstruction Layer ===
        # Output residual with 3x3 conv and add it to input
        self.reconstruct = nn.Conv2d(64, in_channels, kernel_size=3, padding=1)

    def forward(self, x):
        # Save input to add residual later
        input_lr = x

        # Representation path
        x1 = self.conv1(x)
        x2 = self.conv2(x1)
        x3 = self.conv3(x2)
        x4 = self.conv4(x3)
        x5 = self.conv5(x4)

        # Local-global combination using conv3, conv4, conv5
        combined = torch.cat((x3, x4, x5), dim=1)
        fused = self.lgc_merge(combined)

        # Reconstruct residual and add it to the input image
        residual = self.reconstruct(fused)
        output = input_lr + residual

        return output

In [ ]:
def calc_psnr(sr, hr):
    mse = ((sr - hr) ** 2).mean()
    if mse == 0:
        return 100
    return 20 * math.log10(1.0 / math.sqrt(mse))

In [ ]:
def train_model(model, dataloader, device, num_epochs=20):
    criterion = nn.MSELoss()
    optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=1e-4)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=40, gamma=0.1)
    model.train()
    for epoch in range(num_epochs):
        epoch_loss = 0
        for lr, hr in tqdm(dataloader, desc=f"Epoch {epoch+1}/{num_epochs}"):
            lr, hr = lr.to(device), hr.to(device)
            sr = model(lr)
            loss = criterion(sr, hr)
            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=0.4)
            optimizer.step()
            epoch_loss += loss.item()
        scheduler.step()
        print(f"Epoch {epoch+1} Loss: {epoch_loss/len(dataloader):.6f}")
    return model

In [ ]:
def validate_model(model, image_paths, device, scale=3):
    model.eval()
    psnr_scores = []
    ssim_scores = []

    for img_path in tqdm(image_paths, desc="Validating"):
        img = Image.open(img_path).convert('RGB')
        img = np.array(img).astype(np.float32)/255.0
        h, w, _ = img.shape
        h -= h%scale
        w -= w%scale
        img = img[:h,:w,:]

        lr = cv2.resize(img, (w//scale, h//scale), interpolation=cv2.INTER_CUBIC)
        lr_up = cv2.resize(lr, (w, h), interpolation=cv2.INTER_CUBIC)

        lr_tensor = torch.from_numpy(lr_up.transpose(2,0,1)).unsqueeze(0).to(device)
        hr_tensor = torch.from_numpy(img.transpose(2,0,1)).unsqueeze(0).to(device)

        with torch.no_grad():
            sr_tensor = model(lr_tensor).clamp(0,1)

        # Convert to NumPy
        sr = sr_tensor.squeeze().cpu().numpy().transpose(1,2,0)
        hr = hr_tensor.squeeze().cpu().numpy().transpose(1,2,0)

        # Compute metrics
        psnr = calc_psnr(sr, hr)
        ssim = compare_ssim(sr, hr, multichannel=True, data_range=1.0, channel_axis=2)

        psnr_scores.append(psnr)
        ssim_scores.append(ssim)

        # Immediately delete large arrays
        del lr_tensor, hr_tensor, sr_tensor, sr, hr
        torch.cuda.empty_cache()

    print(f"\nAverage PSNR: {np.mean(psnr_scores):.2f} dB")
    print(f"Average SSIM: {np.mean(ssim_scores):.4f}")


In [ ]:
def super_resolve_single_image(model, image_path, device, scale=3):
    model.eval()
    img = Image.open(image_path).convert('RGB')
    img = np.array(img).astype(np.float32)/255.0
    h, w, _ = img.shape
    h -= h%scale
    w -= w%scale
    img = img[:h,:w,:]
    lr = cv2.resize(img, (w//scale, h//scale), interpolation=cv2.INTER_CUBIC)
    lr_up = cv2.resize(lr, (w, h), interpolation=cv2.INTER_CUBIC)
    lr_tensor = torch.from_numpy(lr_up.transpose(2,0,1)).unsqueeze(0).to(device)
    with torch.no_grad():
        sr_tensor = model(lr_tensor).clamp(0,1)
    sr = sr_tensor.squeeze().cpu().numpy().transpose(1,2,0)
    plt.figure(figsize=(12,4))
    plt.subplot(1,3,1); plt.title("Bicubic"); plt.imshow(lr_up); plt.axis("off")
    plt.subplot(1,3,2); plt.title("Super-Resolved"); plt.imshow((sr*255).astype(np.uint8)); plt.axis("off")
    plt.subplot(1,3,3); plt.title("Original"); plt.imshow((img*255).astype(np.uint8)); plt.axis("off")
    plt.tight_layout(); plt.show()

In [ ]:
# Change this path to your uploaded images folder
image_folder = "/kaggle/input/satellite-images/train_sm"

# Load images
image_paths = glob(os.path.join(image_folder, '*.jpg')) + \
              glob(os.path.join(image_folder, '*.png')) + \
              glob(os.path.join(image_folder, '*.jpeg'))

random.shuffle(image_paths)
split = int(0.8 * len(image_paths))
train_paths = image_paths[:split]
val_paths = image_paths[split:]

train_dataset = RemoteSensingDataset(train_paths)
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=2)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = LGCNet().to(device)

# Train
trained_model = train_model(model, train_loader, device)

In [ ]:
import h5py
with h5py.File('model_weights.h5', 'w') as f:
        for k, v in model.state_dict().items():
            f.create_dataset(k, data=v.cpu().numpy()) # Convert tensor to numpy array and move to CPU if on GPU
#model.save('lgcnet_sr.h5')

In [ ]:
# Load the weights from the .h5 file
    loaded_state_dict = {}
    with h5py.File('model_weights.h5', 'r') as f:
        for k in f.keys():
            loaded_state_dict[k] = torch.from_numpy(f[k][:])

    # Create an instance of your model
    loaded_model = MyModel()

    # Load the state_dict into the model
    loaded_model.load_state_dict(loaded_state_dict)
    loaded_model.eval() # Set model to evaluation mode

In [ ]:
# Validate
validate_model(trained_model, val_paths, device)

In [ ]:
# Test on a single image
super_resolve_single_image(trained_model, val_paths[0], device)